<a href="https://colab.research.google.com/github/bigdatastudy-jiyoung/test/blob/master/%EC%82%AC%EC%9D%B4%ED%82%B7%EB%9F%B0_%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


titanic_df=pd.read_csv("train.csv")
titanic_df.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 결측치 확인 및 대치 
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
titanic_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
titanic_df["Age"].fillna(titanic_df["Age"].mean(),inplace=True)
titanic_df["Cabin"].fillna('N',inplace=True)
titanic_df['Embarked'].fillna('N',inplace=True)


In [6]:
titanic_df.isnull().sum().sum()

0

In [7]:
def get_category(age):
  cat=''
  if age<=5:cat="Baby"
  elif age<=12: cat='Child'
  elif age<=18: cat='Teenager'
  elif age<=25: cat='Student'
  elif age<=35: cat='Young Adult'
  elif age<=60: cat='Adult'
  else :cat='Elderly'

  return cat 

  

In [8]:
titanic_df['Age_cat']=titanic_df['Age'].apply(lambda x: get_category(x))

In [9]:
titanic_df['Age_cat'].value_counts()

Young Adult    373
Adult          195
Student        162
Teenager        70
Baby            44
Child           25
Elderly         22
Name: Age_cat, dtype: int64

In [10]:
from sklearn import preprocessing

def encode_features(dataDF):
  features=['Cabin','Sex','Embarked']
  for feature in features:
    le=preprocessing.LabelEncoder()
    le=le.fit(dataDF[feature])
    dataDF[feature]=le.transform(dataDF[feature])
  
  return dataDF

titanic_df=encode_features(titanic_df)
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_cat
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,146,3,Student
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,81,0,Adult
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,146,3,Young Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,55,3,Young Adult
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,146,3,Young Adult


In [11]:
titanic_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Age_cat'],
      dtype='object')

In [12]:
from sklearn.preprocessing import LabelEncoder

#Null 처리함수 
def fillna(df):
  df['Age'].fillna(df['Age'].mean(),inplace=True)
  df['Cabin'].fillna('N',inplace=True)
  df['Embarked'].fillna('N',inplace=True)
  df['Fare'].fillna(0, inplace=True)
  return df

# 머신러닝 알고리즘에 불필요한 속성 제거 
def drop_features(df):
  df.drop(['PassengerId','Name','Ticket'],axis=1, inplace=True)
  return df

# 레이블 인코딩 수행 
def format_features(df):
  df['Cabin']=df['Cabin'].str[:1]
  features=['Cabin','Sex','Embarked']
  for feature in features:
    le=LabelEncoder()
    le.fit(df[feature])
    df[feature]=le.transform(df[feature])
  return df

# 앞에서 설정한 Data Preprocessing 함수 호출

def transform_features(df):
  df=fillna(df)
  df=drop_features(df)
  df=format_features(df)
  return df


In [14]:
#원본데이터 재로딩 하고, feature 데이터셋과  Label 데이터 셋 추출 
titanic_df=pd.read_csv('train.csv')
y_titanic_df=titanic_df['Survived']
x_titanic_df=titanic_df.drop('Survived',axis=1)

x_titanic_df=transform_features(x_titanic_df)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(x_titanic_df, y_titanic_df,
                                                  test_size=0.2, random_state=11)

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [17]:
#모델 객체를 만들고 
dt_clf=DecisionTreeClassifier(random_state=11)
rf_clf=RandomForestClassifier(random_state=11)
lr_clf=LogisticRegression()

#결정트리 학습, 평가 
dt_clf.fit(X_train, y_train)
dt_pred=dt_clf.predict(X_test)
print("결정트리 정확도 : {0:.4f}".format( accuracy_score(y_test, dt_pred)))

# 랜덤포레스트 학습, 평가 
rf_clf.fit(X_train, y_train)
rf_pred=rf_clf.predict(X_test)
print("랜덤포레스트 정확도 :{0:.4f}".format(accuracy_score(y_test, rf_pred)))

# 로지스틱 학습, 평가 
lr_clf.fit(X_train, y_train)
lr_pred=lr_clf.predict(X_test)
print("로지스틱 정확도 : {0:.4f}".format(accuracy_score(y_test, lr_pred)))

결정트리 정확도 : 0.7877
랜덤포레스트 정확도 :0.8547
로지스틱 정확도 : 0.8492


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [20]:
#검증 
from sklearn.model_selection import KFold

def exec_kfold(clf, folds=5):
  # 폴드 수 만큼 예측결과 저장을 위한 리스트 객체 생성.
  kfold=KFold(n_splits=folds)
  scores=[]
  
  #KFold 교차 검증 수행
  for iter_count,(train_index, test_index) in enumerate(kfold.split(x_titanic_df)):
    X_train, X_test=x_titanic_df.values[train_index], x_titanic_df.values[test_index]
    y_train, y_test=y_titanic_df.values[train_index], y_titanic_df.values[test_index]

    clf.fit(X_train, y_train)
    predictions=clf.predict(X_test)
    accuracy=accuracy_score(y_test, predictions)
    scores.append(accuracy)
    print("교차 검증 {0} 정확도  :{0:.4f}".format(iter_count, accuracy))

  mean_score=np.mean(scores) #scores 는 리스트 형태이므로 
  print("평균 정확도 : {0:.4f}".format(mean_score))




In [21]:
exec_kfold(dt_clf, folds=5)

교차 검증 0 정확도  :0.0000
교차 검증 1 정확도  :1.0000
교차 검증 2 정확도  :2.0000
교차 검증 3 정확도  :3.0000
교차 검증 4 정확도  :4.0000
평균 정확도 : 0.7823


In [25]:
from sklearn.model_selection import cross_val_score

scores=cross_val_score(dt_clf,x_titanic_df, y_titanic_df, cv=5)
for iter_count, accuracy in enumerate(scores):
  print("교차검증 {0} 정확도 {1:.4f}".format(iter_count, accuracy))

print("평균 정확도 : {0:.4f}".format(np.mean(scores)))

교차검증 0 정확도 0.7430
교차검증 1 정확도 0.7753
교차검증 2 정확도 0.7921
교차검증 3 정확도 0.7865
교차검증 4 정확도 0.8427
평균 정확도 : 0.7879


In [26]:
from sklearn.model_selection import GridSearchCV
parameters={'max_depth':[2,3,5,10],
            'min_samples_split':[2,3,5],'min_samples_leaf':[1,5,8]}

grid_dclf=GridSearchCV(dt_clf, param_grid=parameters ,scoring='accuracy', cv=5)
grid_dclf.fit(X_train, y_train)

print("최적 파라미터 :", grid_dclf.best_params_)
print("최고 정확도 : {0:.4f}".format(grid_dclf.best_score_))
best_dclf=grid_dclf.best_estimator_

dpredictions=best_dclf.predict(X_test)
accuracy=accuracy_score(y_test, dpredictions)
print('테스트 세트에서의 결정트리 정확도 :{0:.4f}'.format(accuracy))

최적 파라미터 : {'max_depth': 3, 'min_samples_leaf': 5, 'min_samples_split': 2}
최고 정확도 : 0.7992
테스트 세트에서의 결정트리 정확도 :0.8715
